# Check the available cores

In [ ]:
!lscpu --all --extended

CPU NODE SOCKET CORE L1d:L1i:L2:L3 ONLINE
0   0    0      0    0:0:0:0       yes
1   0    0      0    0:0:0:0       yes


In [ ]:
import multiprocessing
print (multiprocessing.cpu_count())

2


# Joblib Common Usage

From https://joblib.readthedocs.io/

In [ ]:
 from math import sqrt

In [ ]:
[sqrt(i ** 2) for i in range(10)]

[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0]

In [ ]:
from joblib import Parallel, delayed

In [ ]:
Parallel(n_jobs=2)(delayed(sqrt)(i ** 2) for i in range(10))

[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0]

# Example for using joblib in Pipeline and Hyperparameter Grid Search

Some code snipets are from Janani Ravi presented in [PluralSight](https://www.pluralsight.com/).

## Generate the data

In [ ]:
from sklearn.datasets import make_classification

In [ ]:
X, y = make_classification(n_samples=10000,
                           n_features=500,
                           n_classes=2,
                           n_redundant=250,
                           random_state=42)

## Set parameters and models

In [ ]:
from sklearn import linear_model, decomposition
from sklearn.pipeline import Pipeline

In [ ]:
logistic = linear_model.LogisticRegression(solver='saga')
pca = decomposition.PCA()

In [ ]:
pipe = Pipeline(steps=[('pca', pca),
                       ('logistic', logistic)])

In [ ]:
grid = dict(pca__n_components=[50, 100, 250],
            logistic__C=[1e-4, 1.0, 1e4],
            logistic__penalty=['l1', 'l2'])

In [ ]:
from sklearn.model_selection import GridSearchCV

## Run with 1 core

In [ ]:
estimator = GridSearchCV(pipe, grid, n_jobs=1, cv=5, error_score='raise')

In [ ]:
%time estimator.fit(X, y)

CPU times: user 2min 46s, sys: 1min 32s, total: 4min 18s
Wall time: 2min 24s


GridSearchCV(cv=5, error_score='raise',
             estimator=Pipeline(memory=None,
                                steps=[('pca',
                                        PCA(copy=True, iterated_power='auto',
                                            n_components=None,
                                            random_state=None,
                                            svd_solver='auto', tol=0.0,
                                            whiten=False)),
                                       ('logistic',
                                        LogisticRegression(C=1.0,
                                                           class_weight=None,
                                                           dual=False,
                                                           fit_intercept=True,
                                                           intercept_scaling=1,
                                                           l1_ratio=None,
                                

In [ ]:
estimator.best_score_

0.8926000000000001

In [ ]:
estimator.best_params_

{'logistic__C': 0.0001, 'logistic__penalty': 'l2', 'pca__n_components': 100}

In [ ]:
estimator.best_estimator_

Pipeline(memory=None,
         steps=[('pca',
                 PCA(copy=True, iterated_power='auto', n_components=100,
                     random_state=None, svd_solver='auto', tol=0.0,
                     whiten=False)),
                ('logistic',
                 LogisticRegression(C=0.0001, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=1,
                                    l1_ratio=None, max_iter=100,
                                    multi_class='auto', n_jobs=None,
                                    penalty='l2', random_state=None,
                                    solver='saga', tol=0.0001, verbose=0,
                                    warm_start=False))],
         verbose=False)

## Run with multiple cores

In [ ]:
estimator = GridSearchCV(pipe, grid, n_jobs=-1, cv=5, error_score='raise')

In [ ]:
%time estimator.fit(X, y)

CPU times: user 2.57 s, sys: 1.18 s, total: 3.75 s
Wall time: 1min 26s


GridSearchCV(cv=5, error_score='raise',
             estimator=Pipeline(memory=None,
                                steps=[('pca',
                                        PCA(copy=True, iterated_power='auto',
                                            n_components=None,
                                            random_state=None,
                                            svd_solver='auto', tol=0.0,
                                            whiten=False)),
                                       ('logistic',
                                        LogisticRegression(C=1.0,
                                                           class_weight=None,
                                                           dual=False,
                                                           fit_intercept=True,
                                                           intercept_scaling=1,
                                                           l1_ratio=None,
                                

In [ ]:
estimator.best_score_

0.8929

In [ ]:
estimator.best_params_

{'logistic__C': 0.0001, 'logistic__penalty': 'l2', 'pca__n_components': 100}

In [ ]:
estimator.best_estimator_

Pipeline(memory=None,
         steps=[('pca',
                 PCA(copy=True, iterated_power='auto', n_components=100,
                     random_state=None, svd_solver='auto', tol=0.0,
                     whiten=False)),
                ('logistic',
                 LogisticRegression(C=0.0001, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=1,
                                    l1_ratio=None, max_iter=100,
                                    multi_class='auto', n_jobs=None,
                                    penalty='l2', random_state=None,
                                    solver='saga', tol=0.0001, verbose=0,
                                    warm_start=False))],
         verbose=False)

# Using Dask for Joblib in scikit-learn

In [ ]:
#%pip install "tornado>=5" 
#%pip install "dask[complete]"
%pip install "cloudpickle>=1.5.0"
!python -m pip install dask distributed --upgrade

  Found existing installation: cloudpickle 1.3.0
    Uninstalling cloudpickle-1.3.0:
      Successfully uninstalled cloudpickle-1.3.0
     |████████████████████████████████| 972kB 7.2MB/s 
     |████████████████████████████████| 716kB 21.8MB/s 
     |████████████████████████████████| 122kB 31.8MB/s 
  Found existing installation: dask 2.12.0
    Uninstalling dask-2.12.0:
      Successfully uninstalled dask-2.12.0
  Found existing installation: distributed 1.25.3
    Uninstalling distributed-1.25.3:
      Successfully uninstalled distributed-1.25.3


May need to restart the runtime after the above pip installation.

In [ ]:
import numpy as np
from dask.distributed import Client

import joblib
from sklearn.datasets import load_digits
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC

client = Client(processes=False)             # create local cluster

In [ ]:
digits = load_digits()

In [ ]:
param_space = {
    'C': np.logspace(-6, 6, 13),
    'gamma': np.logspace(-8, 8, 17),
    'tol': np.logspace(-4, -1, 4),
    'class_weight': [None, 'balanced'],
}

In [ ]:
model = SVC(kernel='rbf')
search = RandomizedSearchCV(model, param_space, cv=3, n_iter=50, verbose=10)

In [ ]:
with joblib.parallel_backend('dask'):
    search.fit(digits.data, digits.target)

Fitting 3 folds for each of 50 candidates, totalling 150 fits
[CV] tol=0.0001, gamma=100.0, class_weight=None, C=1000000.0 .........
[CV] tol=0.1, gamma=1000000.0, class_weight=balanced, C=100.0 ........


[Parallel(n_jobs=-1)]: Using backend DaskDistributedBackend with 2 concurrent workers.


[CV]  tol=0.1, gamma=1000000.0, class_weight=balanced, C=100.0, score=0.100, total=   0.5s
[CV] tol=0.0001, gamma=100.0, class_weight=None, C=1000000.0 .........
[CV]  tol=0.0001, gamma=100.0, class_weight=None, C=1000000.0, score=0.102, total=   0.5s
[CV] tol=0.0001, gamma=100.0, class_weight=None, C=1000000.0 .........


[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    0.5s


[CV]  tol=0.0001, gamma=100.0, class_weight=None, C=1000000.0, score=0.102, total=   0.5s
[CV] tol=0.1, gamma=1000000.0, class_weight=balanced, C=100.0 ........
[CV]  tol=0.0001, gamma=100.0, class_weight=None, C=1000000.0, score=0.100, total=   0.5s
[CV] tol=0.1, gamma=1000000.0, class_weight=balanced, C=100.0 ........


...

[Parallel(n_jobs=-1)]: Done 124 tasks      | elapsed:   26.2s


[CV]  tol=0.1, gamma=100000000.0, class_weight=balanced, C=1000.0, score=0.102, total=   0.4s
[CV] tol=0.0001, gamma=10000000.0, class_weight=balanced, C=10000.0 ..
[CV]  tol=0.0001, gamma=10000000.0, class_weight=balanced, C=10000.0, score=0.100, total=   0.5s
[CV] tol=0.0001, gamma=10000000.0, class_weight=balanced, C=10000.0 ..
[CV]  tol=0.0001, gamma=10000000.0, class_weight=balanced, C=10000.0, score=0.102, total=   0.5s
[CV] tol=0.01, gamma=10.0, class_weight=None, C=1e-05 ................
[CV]  tol=0.0001, gamma=10000000.0, class_weight=balanced, C=10000.0, score=0.102, total=   0.5s
[CV] tol=0.01, gamma=10.0, class_weight=None, C=1e-05 ................
[CV]  tol=0.01, gamma=10.0, class_weight=None, C=1e-05, score=0.100, total=   0.4s
[CV] tol=0.01, gamma=10.0, class_weight=None, C=1e-05 ................
[CV]  tol=0.01, gamma=10.0, class_weight=None, C=1e-05, score=0.102, total=   0.4s
[CV] tol=0.01, gamma=1000.0, class_weight=balanced, C=100.0 ..........
[CV]  tol=0.01, gamma=1

[Parallel(n_jobs=-1)]: Done 141 tasks      | elapsed:   30.2s


[CV]  tol=0.001, gamma=1.0, class_weight=None, C=10000.0, score=0.102, total=   0.5s
[CV] tol=0.001, gamma=100000000.0, class_weight=None, C=100000.0 .....
[CV]  tol=0.001, gamma=1.0, class_weight=None, C=10000.0, score=0.102, total=   0.5s
[CV] tol=0.001, gamma=100000000.0, class_weight=None, C=100000.0 .....
[CV]  tol=0.001, gamma=100000000.0, class_weight=None, C=100000.0, score=0.100, total=   0.5s
[CV] tol=0.001, gamma=100000000.0, class_weight=None, C=100000.0 .....
[CV]  tol=0.001, gamma=100000000.0, class_weight=None, C=100000.0, score=0.102, total=   0.5s
[CV] tol=0.001, gamma=0.01, class_weight=None, C=100.0 ...............
[CV]  tol=0.001, gamma=100000000.0, class_weight=None, C=100000.0, score=0.102, total=   0.5s
[CV] tol=0.001, gamma=0.01, class_weight=None, C=100.0 ...............
[CV]  tol=0.001, gamma=0.01, class_weight=None, C=100.0, score=0.664, total=   0.5s
[CV] tol=0.001, gamma=0.01, class_weight=None, C=100.0 ...............
[CV]  tol=0.001, gamma=0.01, class_wei

[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed:   32.3s finished
